In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import sqlite3 as sql


In [2]:
os.listdir("../KAGGLE/DATA")

['diamonds_test.csv', 'diamonds_train.db', 'sample_submission.csv']

In [3]:
database = "../KAGGLE/DATA/diamonds_train.db"
connection = sql.connect(database)

In [4]:
query = '''SELECT tr.index_id, c.clarity, cl.color, ct.cut, dm.depth, dm.'table', dm.x, dm.y, dm.z, tr.price, ci.city, tr.carat
FROM diamonds_properties pr
   INNER JOIN diamonds_clarity c ON pr.clarity_id = c.clarity_id
   INNER JOIN diamonds_color cl on pr.color_id = cl.color_id
   INNER JOIN diamonds_cut ct on pr.cut_id = ct.cut_id
   INNER JOIN diamonds_dimensions dm on pr.index_id = dm.index_id
   INNER JOIN diamonds_transactional tr on pr.index_id = tr.index_id
   INNER JOIN diamonds_city ci on tr.city_id = ci.city_id'''

In [5]:
diamonds = pd.read_sql_query(query,connection)
diamonds.head()

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02


DATA CLEANING DIAMONDS


In [6]:
diamonds.shape


(40455, 12)

In [7]:
diamonds.dtypes


index_id     object
clarity      object
color        object
cut          object
depth       float64
table       float64
x           float64
y           float64
z           float64
price         int64
city         object
carat       float64
dtype: object

In [8]:
#COMO PODEMOS VER NO NOS FALTAN FILAS EN EL DATASET, LO PODEMOS VER YA QUE EL SHAPE Y EL COUNT AL HACER EL DESCRIBE() ES EL MISMO, POR LO QUE NO NOS FALTAN DATOS EN NINGUNA COLUMNA

diamonds.describe()


,depth,table,x,y,z,price,carat
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,61.752841,57.446133,5.729392,5.732819,3.537154,3928.444469,0.797706
std,1.431725,2.233535,1.124453,1.146650,0.697062,3992.416147,0.475544
min,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000,0.200000
25%,61.000000,56.000000,4.710000,4.720000,2.910000,945.000000,0.400000
50%,61.800000,57.000000,5.690000,5.710000,3.520000,2397.000000,0.700000
75%,62.500000,59.000000,6.540000,6.540000,4.035000,5331.000000,1.040000
max,79.000000,95.000000,10.230000,58.900000,8.060000,18823.000000,4.500000


In [9]:
#COMPROBACION DE LO EXPLICADO ANTERIORMENTE, NO TENEMOS NULOS.
pd.isnull(diamonds).sum()

index_id    0
clarity     0
color       0
cut         0
depth       0
table       0
x           0
y           0
z           0
price       0
city        0
carat       0
dtype: int64

In [10]:
#OTRA COMPROBACION DE LO ANTERIOR
pd.isnull(diamonds).values.ravel().sum()

0

CREAMOS LAS VARIBALES DUMMIES

In [11]:
#VARIABLE DUMMY PARA CLARITY
dummy_clarity = pd.get_dummies(diamonds["clarity"], prefix= "clarity")
dummy_clarity.head()

,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0


In [12]:
dummy_color = pd.get_dummies(diamonds["color"], prefix= "color")
dummy_color.head()

,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0


In [13]:
dummy_cut = pd.get_dummies(diamonds["cut"], prefix= "cut")
dummy_cut.head()

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,0,0,0,1,0
1,0,0,0,0,1
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0


In [14]:
dummy_city = pd.get_dummies(diamonds["city"], prefix= "city")
dummy_city.head()

,city_Amsterdam,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0


In [15]:
#HACEMOS EL CONCAT DE LAS COLUMNAS DUMMIES QUE HE CREADO CON EL ANTIGUO DATASET QUE TENIAMOS

diamonds_dummies = pd.concat([diamonds, dummy_clarity,dummy_color,dummy_cut,dummy_city], axis=1)
lista_columnas = diamonds_dummies.columns.values.tolist()
lista_columnas




['index_id',
 'clarity',
 'color',
 'cut',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'price',
 'city',
 'carat',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'city_Amsterdam',
 'city_Antwerp',
 'city_Dubai',
 'city_Kimberly',
 'city_Las Vegas',
 'city_London',
 'city_Luxembourg',
 'city_Madrid',
 'city_New York City',
 'city_Paris',
 'city_Surat',
 'city_Tel Aviv',
 'city_Zurich']

In [16]:
#ELIMINAMOS LAS COLUMNAS DE LAS CUALES HEMOS SACADO LAS VARIABLES DUMMIES

diamonds_dummies = diamonds_dummies.drop(['clarity'], axis = 1)


In [17]:
diamonds_dummies

,index_id,color,cut,depth,table,x,y,z,price,city,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,...,0,0,0,0,0,0,0,0,0,0
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,...,1,0,0,0,0,0,0,0,0,0
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,...,0,1,0,0,0,0,0,0,0,0
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,...,1,0,0,0,0,0,0,0,0,0
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,G,Ideal,62.7,57.0,7.10,7.04,4.43,10070,Antwerp,...,0,0,0,0,0,0,0,0,0,0
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,F,Good,57.1,60.0,8.31,8.25,4.73,12615,Madrid,...,0,0,0,0,1,0,0,0,0,0
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,H,Ideal,62.7,56.0,6.37,6.42,4.01,5457,Kimberly,...,1,0,0,0,0,0,0,0,0,0
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,J,Ideal,61.9,54.3,4.45,4.47,2.76,456,Kimberly,...,1,0,0,0,0,0,0,0,0,0


DIVIDIMOS TODO EL DATASET EN LAS FEATURES Y EN LA PREDICCION

In [18]:
 diamonds_dummies_columns = diamonds_dummies.columns.values.tolist()
 diamonds_dummies_columns

['index_id',
 'color',
 'cut',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'price',
 'city',
 'carat',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'city_Amsterdam',
 'city_Antwerp',
 'city_Dubai',
 'city_Kimberly',
 'city_Las Vegas',
 'city_London',
 'city_Luxembourg',
 'city_Madrid',
 'city_New York City',
 'city_Paris',
 'city_Surat',
 'city_Tel Aviv',
 'city_Zurich']

In [19]:
features = ['depth',
 'table',
 'x',
 'y',
 'z',
 'carat',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'city_Amsterdam',
 'city_Antwerp',
 'city_Dubai',
 'city_Kimberly',
 'city_Las Vegas',
 'city_London',
 'city_Luxembourg',
 'city_Madrid',
 'city_New York City',
 'city_Paris',
 'city_Surat',
 'city_Tel Aviv',
 'city_Zurich']

In [20]:
#CREAMOS NUESTRA X y NUESTRA Y

x = diamonds_dummies[features]
y = diamonds_dummies['price']


In [21]:
len(features)

39

#METODO QUE NOS DICE CUALES SON LAS MEJORES VARIABLES QUE TENEMOS
from sklearn.feature_selection import RFE 
from sklearn.svm import SVR 

estimator = SVR(kernel ="linear")
selector = RFE (estimator,n_features_to_select= 10, step=1) #MUCHO CUIDADO PARA PONER CUANTAS FEATURES QUEREMOS QUE TENGA NUESTRO MODELO TENEMOS QUE PONER EL PARAMETRO: n_features_to_select= el numero de features que queremos. 
selector = selector.fit(x,y)

VAMOS A CREAR EL MODELO DE REGRESION LINEAL CON SCIKIT-LEARN


In [22]:
from sklearn.linear_model import LinearRegression

In [25]:
#EN ESTA CELDA PODREMOS PONER Y QUITAR LAS FEATURES QUE NOSOTROS QUERAMOS PARA NUESTRO MODELO
x_pred = x

In [26]:
lm = LinearRegression()
lm.fit(x_pred, y)

LinearRegression()

In [27]:
#PODEMOS SACAR ALFA, EL NUMERITO SUELTO
lm.intercept_

489447083104890.4

In [28]:
#Y CON ESTE METODO PODEMOS SACAR LOS COEFICIENTES QUE ACOMPAÑAN A CADA UNA DE NUESTRAS FEATURES
lm.coef_

array([-6.34665640e+01, -2.78603564e+01, -9.90990411e+02,  2.99459681e+00,
       -9.38474687e+01,  1.12879150e+04,  1.34515824e+14,  1.34515824e+14,
        1.34515824e+14,  1.34515824e+14,  1.34515824e+14,  1.34515824e+14,
        1.34515824e+14,  1.34515824e+14, -7.23553481e+13, -7.23553481e+13,
       -7.23553481e+13, -7.23553481e+13, -7.23553481e+13, -7.23553481e+13,
       -7.23553481e+13, -5.11298818e+14, -5.11298818e+14, -5.11298818e+14,
       -5.11298818e+14, -5.11298818e+14, -4.03087410e+13, -4.03087410e+13,
       -4.03087410e+13, -4.03087410e+13, -4.03087410e+13, -4.03087410e+13,
       -4.03087410e+13, -4.03087410e+13, -4.03087410e+13, -4.03087410e+13,
       -4.03087410e+13, -4.03087410e+13, -4.03087410e+13])

In [29]:
#ESTE METODO NOS SACARA EL VALOR DE R2 AJUSRTADO.

lm.score(x_pred, y)

0.9206043844400313

In [30]:
list(zip(features,lm.coef_))

[('depth', -63.46656395521507),
 ('table', -27.86035640852098),
 ('x', -990.9904107898701),
 ('y', 2.9945968074634566),
 ('z', -93.84746870714463),
 ('carat', 11287.915039680442),
 ('clarity_I1', 134515824148551.33),
 ('clarity_IF', 134515824153950.88),
 ('clarity_SI1', 134515824152300.31),
 ('clarity_SI2', 134515824151340.53),
 ('clarity_VS1', 134515824153208.73),
 ('clarity_VS2', 134515824152900.4),
 ('clarity_VVS1', 134515824153641.52),
 ('clarity_VVS2', 134515824153575.56),
 ('color_D', -72355348135279.9),
 ('color_E', -72355348135484.58),
 ('color_F', -72355348135551.73),
 ('color_G', -72355348135762.03),
 ('color_H', -72355348136257.25),
 ('color_I', -72355348136724.47),
 ('color_J', -72355348137651.42),
 ('cut_Fair', -511298818156247.06),
 ('cut_Good', -511298818155668.3),
 ('cut_Ideal', -511298818155424.8),
 ('cut_Premium', -511298818155489.8),
 ('cut_Very Good', -511298818155522.8),
 ('city_Amsterdam', -40308740959629.64),
 ('city_Antwerp', -40308740959711.305),
 ('city_Dubai'

## NUESTRO MODELO
489447083104890.4 +
diamonds_dummies['depth']* -63.46656395521507 +
diamonds_dummies['table']* -27.86035640852098 +
 diamonds_dummies['x']* -990.9904107898701 +
 diamonds_dummies['y']* 2.9945968074634566 +
 diamonds_dummies['z']* -93.84746870714463 +
 diamonds_dummies['carat']* 11287.915039680442 +
 diamonds_dummies['clarity_I1']* 134515824148551.33 +
 diamonds_dummies['clarity_IF]'* 134515824153950.88 +
 diamonds_dummies['clarity_SI1']* 134515824152300.31 +
 diamonds_dummies['clarity_SI2']* 134515824151340.53 +
 diamonds_dummies['clarity_VS1']* 134515824153208.73 +
 diamonds_dummies['clarity_VS2']* 134515824152900.4 +
 diamonds_dummies['clarity_VVS1']* 134515824153641.52 +
 diamonds_dummies['clarity_VVS2']* 134515824153575.56 +
 diamonds_dummies['color_D']* -72355348135279.9 +
 diamonds_dummies['color_E']* -72355348135484.58 +
 diamonds_dummies['color_F']* -72355348135551.73 +
 diamonds_dummies['color_G']* -72355348135762.03 +
 diamonds_dummies['color_H']* -72355348136257.25 +
 diamonds_dummies['color_I']* -72355348136724.47 +
 diamonds_dummies['color_J']* -72355348137651.42 +
 diamonds_dummies['cut_Fair']* -511298818156247.06 +
 diamonds_dummies['cut_Good']* -511298818155668.3 +
 diamonds_dummies['cut_Ideal']* -511298818155424.8 +
 diamonds_dummies['cut_Premium']* -511298818155489.8 +
 diamonds_dummies['cut_Very Good']* -511298818155522.8 +
 diamonds_dummies['city_Amsterdam']* -40308740959629.64 +
 diamonds_dummies['city_Antwerp']* -40308740959711.305 +
 diamonds_dummies['city_Dubai']* -40308740959669.734 +
 diamonds_dummies['city_Kimberly']* -40308740959675.42 +
 diamonds_dummies['city_Las Vegas']* -40308740959665.06 +
 diamonds_dummies['city_London']* -40308740959716.59 +
 diamonds_dummies['city_Luxembourg']* -40308740959684.97 +
 diamonds_dummies['city_Madrid']* -40308740959647.52 +
 diamonds_dummies['city_New York City']* -40308740959707.555 +
 diamonds_dummies['city_Paris']* -40308740959688.484 +
 diamonds_dummies['city_Surat']* -40308740959693.195 +
 diamonds_dummies['city_Tel Aviv']* -40308740959670.31 +
 diamonds_dummies['city_Zurich']* -40308740959638.28


In [32]:
diamonds_dummies["Prediction"] = 489447083104890.4 + diamonds_dummies['depth']* -63.46656395521507 + diamonds_dummies['table']* -27.86035640852098 + diamonds_dummies['x']* -990.9904107898701 +diamonds_dummies['y']* 2.9945968074634566 +diamonds_dummies['z']* -93.84746870714463 +diamonds_dummies['carat']* 11287.915039680442 +diamonds_dummies['clarity_I1']* 134515824148551.33 +diamonds_dummies['clarity_IF']* 134515824153950.88 +diamonds_dummies['clarity_SI1']* 134515824152300.31 +diamonds_dummies['clarity_SI2']* 134515824151340.53 +diamonds_dummies['clarity_VS1']* 134515824153208.73 +diamonds_dummies['clarity_VS2']* 134515824152900.4 +diamonds_dummies['clarity_VVS1']* 134515824153641.52 +diamonds_dummies['clarity_VVS2']* 134515824153575.56 +diamonds_dummies['color_D']* -72355348135279.9 +diamonds_dummies['color_E']* -72355348135484.58 +diamonds_dummies['color_F']* -72355348135551.73 +diamonds_dummies['color_G']* -72355348135762.03 +diamonds_dummies['color_H']* -72355348136257.25 +diamonds_dummies['color_I']* -72355348136724.47 +diamonds_dummies['color_J']* -72355348137651.42 +diamonds_dummies['cut_Fair']* -511298818156247.06 +diamonds_dummies['cut_Good']* -511298818155668.3 +diamonds_dummies['cut_Ideal']* -511298818155424.8 + diamonds_dummies['cut_Premium']* -511298818155489.8 +diamonds_dummies['cut_Very Good']* -511298818155522.8 +diamonds_dummies['city_Amsterdam']* -40308740959629.64 +diamonds_dummies['city_Antwerp']* -40308740959711.305 +diamonds_dummies['city_Dubai']* -40308740959669.734 +diamonds_dummies['city_Kimberly']* -40308740959675.42 +diamonds_dummies['city_Las Vegas']* -40308740959665.06 +diamonds_dummies['city_London']* -40308740959716.59 +diamonds_dummies['city_Luxembourg']* -40308740959684.97 +diamonds_dummies['city_Madrid']* -40308740959647.52 +diamonds_dummies['city_New York City']* -40308740959707.555 +diamonds_dummies['city_Paris']* -40308740959688.484 +diamonds_dummies['city_Surat']* -40308740959693.195 +diamonds_dummies['city_Tel Aviv']* -40308740959670.31 +diamonds_dummies['city_Zurich']* -40308740959638.28

In [33]:
diamonds_dummies.head()

,index_id,color,cut,depth,table,x,y,z,price,city,...,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich,Prediction
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,...,0,0,0,0,0,0,0,0,0,5915.015625
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,...,0,0,0,0,0,0,0,0,0,-194.734375
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,...,1,0,0,0,0,0,0,0,0,2854.625000
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,...,0,0,0,0,0,0,0,0,0,680.515625
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,...,0,0,0,0,0,0,0,0,0,5522.203125


In [34]:
SSD = np.sum((diamonds_dummies["Prediction"] - diamonds_dummies["price"])**2)

In [35]:
RSE = np.sqrt(SSD/(len(diamonds_dummies)- len(features) -1))

In [36]:
price_mean = np.mean(diamonds_dummies["price"])
price_mean

3928.444469163268

In [37]:
ERROR = (RSE/price_mean)*100 
ERROR
print("TENEMOS UN ERROR DEL " + str(ERROR) + "%")

TENEMOS UN ERROR DEL 28.649871271469557%
